In [1]:
import pandas as pd
from collections import defaultdict
import re, json
from tabulate import tabulate
import numpy as np

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [3]:
students = gc.open_by_key('1vrK1z1COcaa9qXiddxLn1NjvC0T9vMjcKA4W7P9gSds') #2024 link, replace with 2025 link later
students = students.worksheet('Form Responses 1').get_all_values()
students_df = pd.DataFrame(students[1:], columns=students[0])

In [4]:
student_name_col = [col for col in students_df.columns if 'full name' in col.lower()][0]
student_year_col = [col for col in students_df.columns if 'year' in col.lower()][0]
student_major_col = [col for col in students_df.columns if 'major' in col.lower()][0]
student_role_col = [col for col in students_df.columns if 'position' in col.lower()][0]
student_intl_col = [col for col in students_df.columns if 'international student' in col.lower()][0]
student_year_col = [col for col in students_df.columns if 'year' in col.lower()][0]
print(student_year_col)

students = students_df[[student_name_col, student_year_col, student_major_col, student_role_col, student_intl_col]]
students.replace("", np.nan, inplace=True)
students = students.dropna(how='all')
students = students.applymap(lambda x: x.strip() if isinstance(x, str) else x)

Year at UCLA


<ipython-input-4-da88d9506f7c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  students.replace("", np.nan, inplace=True)
<ipython-input-4-da88d9506f7c>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  students = students.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [5]:
companies = gc.open_by_key('1hL4RKNKZEGOJbzIklnyUwD_tEOp5uoEzzfgwtNDN1o0')
companies = companies.worksheet('Form Responses 1').get_all_values()
companies_df = pd.DataFrame(companies[1:], columns = companies[0])

In [6]:
companies_name_col = [col for col in companies_df.columns if 'name' in col.lower() ][0]
companies_role_hire_col = [col for col in companies_df.columns if 'position' in col.lower() ][0]
companies_major_hire_col = [col for col in companies_df.columns if 'majors' in col.lower() ][0]
companies_intl_sponsor_col = [col for col in companies_df.columns if 'citizenship' in col.lower()][0]


companies = companies_df[[companies_name_col, companies_role_hire_col, companies_major_hire_col, companies_intl_sponsor_col]]
companies.replace("", np.nan, inplace=True)
companies = companies.dropna(how='all')
companies = companies.applymap(lambda x: x.strip() if isinstance(x, str) else x)

<ipython-input-6-ccae788873ee>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies.replace("", np.nan, inplace=True)
<ipython-input-6-ccae788873ee>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  companies = companies.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [7]:
rep_names = gc.open_by_key('1Mg1TYEsNppf520WjaRq1rzEVp3J9jOtotZb7yOAPeCg')
rep_names = rep_names.worksheet('Form Responses 1').get_all_values()
rep_names_df = pd.DataFrame(rep_names[1:], columns = rep_names[0])


first_rep_column = [col for col in rep_names_df.columns if 'representative #1: name' in col.lower()][0]
second_rep_column = [col for col in rep_names_df.columns if 'representative #2: name' in col.lower()][0]


rep_names = rep_names_df[[first_rep_column, second_rep_column]]
rep_names.replace("", np.nan, inplace=True)
rep_names.dropna(how='all', inplace=True)
rep_names.dropna(axis=1, how='all', inplace=True)
rep_names = rep_names.applymap(lambda x: x.strip() if isinstance(x, str) else x)



<ipython-input-7-03007cc927e4>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  rep_names.replace("", np.nan, inplace=True)
<ipython-input-7-03007cc927e4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rep_names.replace("", np.nan, inplace=True)
<ipython-input-7-03007cc927e4>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rep_names.dropna(how='all', inplace=True)
<ipython-input-7-0300

In [8]:
matches = {}
matched_students = set()

#set of students who didnt get their any of preferences
no_preferences_student = []

info_dict = {}

for _,company in companies.iterrows():
  info_dict[company[companies_name_col].strip()] = []
  matches[company[companies_name_col].strip()] = []

print(info_dict)
print(matches)

{'PPG Industries': [], 'Cadence': [], 'Los Angeles Department of Water and Power (LADWP)': [], 'DIRECTV': [], 'Siemens DISW': [], 'The Aerospace Corporation': [], 'Northrop Grumman': [], 'RCAM Technologies': [], 'SpaceX': [], 'PSC Biotech, Inc.': [], 'Los Angeles County Sanitation Districts': [], 'Howmet Aerospace': [], 'LA Sanitation & Environment (City of Los Angeles)': [], 'Medtronic': [], 'Lockheed Martin': [], 'Applied Medical': [], 'Gilead Sciences': [], 'Disney Research': [], 'Varda Space Industries': [], 'Southern California Edison': [], 'Boeing': [], 'Fluor Enterprises, Inc.': [], 'SoCalGas': [], 'Lightship RV': [], 'Qvest.US': [], 'Accenture': [], 'ATA Engineering': [], 'Takeda': [], 'L3Harris Technologies': [], 'PepsiCo': [], 'Parker Aerospace': [], 'FM Global': [], 'Divergent': []}
{'PPG Industries': [], 'Cadence': [], 'Los Angeles Department of Water and Power (LADWP)': [], 'DIRECTV': [], 'Siemens DISW': [], 'The Aerospace Corporation': [], 'Northrop Grumman': [], 'RCAM Te

In [9]:
# for EWI committee! (or just other manually added matches)

while True:
    student_name = input("Enter student name (or type 'quit' to exit): ").strip()
    if student_name.lower() == 'quit':
        break
    company_name = input("Enter company name: ").strip()
    matched_students.add(student_name)
    matches[company_name].append(student_name)

Enter student name (or type 'quit' to exit): quit


In [10]:
priority_lists = {}
for index, row in students_df.iterrows():
  for ranking in ['First', 'Second', 'Third', 'Fourth', 'Fifth']:
    company = [col for col in students_df.columns[17-1 : 49] if ranking in row[col]] #16-49 contain the company choices
    if company:
      company = re.search(r'\[(.*?)\]', company[0])
      company = company.group(0).strip("[]")
      if row[student_name_col] in priority_lists:
          priority_lists[row[student_name_col]].append(company.strip())
      else:
          priority_lists[row[student_name_col]] = [company.strip()]

print(json.dumps(priority_lists, indent=4))

{
    "Claris Chan": [
        "Northrop Grumman",
        "Lockheed Martin",
        "Boeing",
        "Varda Space Industries",
        "L3Harris Technologies"
    ],
    "Anushka Sikdar": [
        "Disney Research",
        "SpaceX",
        "Varda Space Industries"
    ],
    "Meryl Mathew": [
        "SpaceX",
        "Accenture",
        "Disney Research",
        "RCAM Technologies",
        "DIRECTV"
    ],
    "Sophie Zhu": [
        "Cadence",
        "Siemens Digital Industries Software",
        "Disney Research",
        "Boeing",
        "SpaceX"
    ],
    "Kenneth Wan": [
        "SpaceX",
        "Boeing",
        "Cadence",
        "Siemens Digital Industries Software",
        "Accenture"
    ],
    "Aashna Sibal": [
        "SpaceX",
        "Qvest.US",
        "Disney Research",
        "Medtronic",
        "DIRECTV"
    ],
    "Isabella Pascual": [
        "Medtronic",
        "Applied Medical",
        "PSC Biotech, Inc.",
        "Gilead Sciences",
        "Tak

In [11]:
for student_name, company_list in priority_lists.items():
    priority_lists[student_name] = ["Siemens DISW" if company == 'Siemens Digital Industries Software' else company for company in company_list]

In [12]:
def major_match():
    global info_dict

    students_by_major = students.groupby(student_major_col)[student_name_col].apply(list).to_dict()
    consolidated_dict = {}
    for major, names in students_by_major.items():
        changed = False
        if any(keyword in major.lower() for keyword in ["dse", "data", "computing", "computation", "computer"]):
            changed = True
            if "Computer Science" not in consolidated_dict:
                consolidated_dict["Computer Science"] = []
            consolidated_dict["Computer Science"].extend(names)
        if any(keyword in major.lower() for keyword in ["undeclared"]):
            changed = True
            if "Undeclared" not in consolidated_dict:
                consolidated_dict["Undeclared"] = []
            consolidated_dict["Undeclared"].extend(names)
        if any(keyword in major.lower() for keyword in ["math"]):
            changed = True
            if "Mathematics" not in consolidated_dict:
                consolidated_dict["Mathematics"] = []
            consolidated_dict["Mathematics"].extend(names)
        if any(keyword in major.lower() for keyword in ["bio"]):
            changed = True
            if "Bioengineering" not in consolidated_dict:
                consolidated_dict["Bioengineering"] = []
            consolidated_dict["Bioengineering"].extend(names)
        if not changed:
            consolidated_dict[major] = names
    students_by_major = consolidated_dict

    for _, company in companies.iterrows():
        company_name = company[companies_name_col]
        company_major_str = company[companies_major_hire_col]
        company_majors = [major.strip() for major in company_major_str.split(', ')]

        for major in company_majors:
          if major in students_by_major.keys():
            info_dict[company_name].extend(students_by_major[major])

    return info_dict

In [13]:
def intl_match():
    global info_dict

    students_by_intl = students.groupby(student_intl_col)[student_name_col].apply(list).to_dict()

    for _, company in companies.iterrows():
        company_name = company[companies_name_col]
        if any(keyword in company[companies_intl_sponsor_col].lower() for keyword in ["citizenship required", "yes, we require citizenship", "yes"]): # company requires citizenship
          info_dict[company_name] = [student for student in info_dict[company_name] if student not in students_by_intl["Yes"]] # remove from company potential students if intl students

    return info_dict

In [14]:
from difflib import SequenceMatcher

#basically goes through the string finds common substring, extracts it

def find_similar_strings(s1, s2, min_length = 2):
  seq_match = SequenceMatcher(None, s1, s2)
  match = seq_match.find_longest_match(0, len(s1), 0, len(s2))

  common_string = []
  while match.size >= min_length:
    one_common_string = s1[match.a : match.a + match.size]
    common_string.append(one_common_string)
    s1 = s1.replace(one_common_string, "")
    s2 = s2.replace(one_common_string, "")
    seq_match = SequenceMatcher(None, s1, s2)
    match = seq_match.find_longest_match(0, len(s1), 0, len(s2))
  return common_string

In [15]:
'''''for each company, see if each student is a fit for that company and put it in a list. Iterate that student list in order to make it easier for FCFS '''
students[student_role_col] = students[student_role_col].str.lower()
companies[companies_role_hire_col] = companies[companies_role_hire_col].str.lower()
students[student_role_col] = students[student_role_col].replace('full time', 'full-time')
companies[companies_role_hire_col] = companies[companies_role_hire_col].str.replace('summer internship' , 'internship')
companies[companies_role_hire_col] = companies[companies_role_hire_col].str.replace('spring/ongoing interns' , 'internship')
# print(companies[companies_role_hire_col])
# print(students[student_role_col])

In [16]:
#role sort

def role_match():
  global info_dict
  for _, company in companies.iterrows():
    for _, student in students.iterrows():
      student_name = student[student_name_col]
      company_name = company[companies_name_col]
      similar_strings = find_similar_strings(company[companies_role_hire_col], student[student_role_col])

      if company_name == "Accenture": # just accounting for Accenture's different hiring policy
        continue

      #print(company[companies_name_col] + company[companies_role_hire_col] + ' ' + student_name + ' ' + student[student_role_col] + ' ' + str(similar_strings))
      #if student_role in similar_strings:
        #print('true' + ' ' + company_name + student_role + str(similar_strings))
      if student[student_role_col] not in similar_strings:
        if student_name in info_dict[company_name]:
          info_dict[company_name].remove(student_name)
      before_phd_info_dict = info_dict

      #checks for phd masters --- this might be redundant because i think the code above naturally already does that
      if any(keyword in student[student_year_col].lower() for keyword in ["grad student - master's", "grad student - phd"]):
        #idea checks if the student is in any companies NOT offering full time and pulls them
        if 'full-time' not in company[companies_role_hire_col]:
          if student_name in info_dict[company_name]:
            info_dict[company_name].remove(student_name)
        #if grad students checks internship
        elif student[student_role_col] == 'internship' and student_name not in info_dict[company_name]:
          if student[student_major_col] in company[companies_major_hire_col]: #makes sure companies takes their major
            if student[student_intl_col].lower() == 'no' or (student[student_intl_col].lower() == 'yes' and 'not require citizenship or a green card' in company[companies_intl_sponsor_col].lower()): #make sure companies takes international:
              info_dict[company_name].append(student_name)

  return info_dict

In [17]:
major_match()
intl_match()
role_match()
print("Base matching done.")

Base matching done.


In [18]:
# def clean_strings_in_dict(d):
#     if isinstance(d, dict):
#         return {k.strip(): clean_strings_in_dict(v) for k, v in d.items()}
#     elif isinstance(d, list):
#         return [clean_strings_in_dict(i) for i in d]
#     elif isinstance(d, str):
#         return d.strip()
#     else:
#         return d

# matches = clean_strings_in_dict(matches)
# info_dict = clean_strings_in_dict(info_dict)

In [19]:
def fcfs():
    global matches, matched_students, info_dict, no_preferences_student
    for students_list in matches.values():
      for student in students_list:
          matched_students.add(student)

    company_major_counts = {}

    for company_index, company in companies.iterrows():
        company_name = company[companies_name_col]
        company_major_str = company[companies_major_hire_col]
        company_majors = [major.strip() for major in company_major_str.split(', ')]
        company_major_counts[company_name] = {major: 0 for major in company_majors}

    #*put all students in the set
    for _, student in students.iterrows():
      student_name = student[student_name_col]
      no_preferences_student.append(student_name)


    for _, student in students.iterrows():
        student_name = student[student_name_col]
        student_major = student[student_major_col]

        if student_name not in priority_lists:
            continue

        priority_list = priority_lists[student_name]

        for company_name in priority_list:
            company = companies.loc[companies[companies_name_col] == company_name]
            if not company.empty:
              if student_name in info_dict[company_name]:
                  company_major_str = company[companies_major_hire_col].iloc[0]
                  company_majors = [major.strip() for major in company_major_str.split(', ')]

                  current_major_count = company_major_counts[company_name].get(student_major, 0)

                  if len(matches[company_name]) <= 7 and (current_major_count + 1) <= 4:
                      matches[company_name].append(student_name)
                      matched_students.add(student_name)
                      company_major_counts[company_name][student_major] = current_major_count + 1
                      #*if student is matched, remove them for the set
                      no_preferences_student.remove(student_name)
                      break

    return matches, company_major_counts

matches, counts = fcfs()
# print(counts)
print(json.dumps(matches, indent=4))
print(no_preferences_student)

{
    "PPG Industries": [
        "Althea Mendoza",
        "Khanakk Garg",
        "Abigail Jin"
    ],
    "Cadence": [
        "Sophie Zhu",
        "Lindsay Qin",
        "Ryan Yeo",
        "Summer Dixon",
        "Roocha Thatte",
        "Renuka Bhusari",
        "Victor Lee",
        "James He"
    ],
    "Los Angeles Department of Water and Power (LADWP)": [
        "Elisha Reyes",
        "Sidney Sabater",
        "Tina Fang",
        "Jason Wright"
    ],
    "DIRECTV": [
        "Riya Gupta",
        "Sharon Chen",
        "Angela Tan",
        "Colleen Lam",
        "Michael Bunte",
        "Anusha Singhai",
        "Jasmine Wongphatarakul",
        "Corey Shen"
    ],
    "Siemens DISW": [
        "Leah George",
        "Khush Parikh",
        "Samuel Lu",
        "Alex Yoon",
        "May He",
        "Tiffany Chen",
        "Ashley Cheng",
        "Tiffany Orian"
    ],
    "The Aerospace Corporation": [
        "Louis Law",
        "Nandana Venkitesh",
        "Anika Mi

In [20]:
def more_matching():
    global matches, matched_students, info_dict, no_preferences_student, counts, priority_lists, students

    rematch_count = {}

    i = 0
    while i < len(no_preferences_student):
        student_name = no_preferences_student[i]

        if student_name not in priority_lists:
            i += 1
            continue

        # Check if the student is in the first 60% who registered
        student_index = students.index[students[student_name_col] == student_name].tolist()
        if not student_index or student_index[0] >= int(len(students) * 0.6):
            i += 1
            continue

        priority_list = reversed(priority_lists[student_name])
        match_made = False

        for company_name in priority_list:
            for other_student in reversed(matches[company_name]):
                priority_list_other = priority_lists[other_student]
                other_student_rank = priority_list_other.index(company_name) if company_name in priority_list_other else len(priority_list_other) + 1
                other_student_index = students.index[students[student_name_col] == other_student].tolist()

                # Check if the student is in the first 40% who registered and don't replace if so
                if not other_student_index or other_student_index[0] <= int(len(students) * 0.4):
                    continue

                # Check rematch count and ensure it's not exceeded 3
                if rematch_count.get(other_student, 0) >= 3:
                    continue

                # Check that we are only replacing students that ranked the company 3rd or 4th
                if other_student_rank == 2 or other_student_rank == 3:
                    matches[company_name].remove(other_student)
                    matched_students.remove(other_student)
                    matches[company_name].append(student_name)
                    matched_students.add(student_name)
                    info_dict[company_name].append(student_name)

                    no_preferences_student.append(other_student)
                    no_preferences_student.pop(i)

                    rematch_count[other_student] = rematch_count.get(other_student, 0) + 1
                    rematch_count[student_name] = rematch_count.get(student_name, 0) + 1

                    print(f"{student_name} matched with {company_name}, replacing {other_student}.")

                    match_made = True
                    break

            if match_made:
                break

        if not match_made:
            i += 1

more_matching()

Charlotte Schmitt matched with PSC Biotech, Inc., replacing Alex Chen.
Angela Yang matched with Applied Medical, replacing Jamie Weisel.
Bryan Kwan matched with Lockheed Martin, replacing Hana Chloe Yoon.
Chinenye Eneh matched with L3Harris Technologies, replacing Zeckria Kamrany.
Sheyla Portillo matched with Southern California Edison, replacing Emily Nham.
Anna Moulds matched with PSC Biotech, Inc., replacing Kaimana Lum.
Maya Chatrathi matched with Lockheed Martin, replacing Bryan Kwan.
Pranathi Kollolli matched with Southern California Edison, replacing Sheyla Portillo.
Chloe Cheng matched with Lockheed Martin, replacing Kai Wang.
Esha Sidhu matched with Lockheed Martin, replacing Chloe Cheng.
Edmund Leong matched with ATA Engineering, replacing Hannah Yared.
Jamie Weisel matched with Applied Medical, replacing Angela Yang.
Hana Chloe Yoon matched with RCAM Technologies, replacing Johanne Moresco.
Zeckria Kamrany matched with L3Harris Technologies, replacing Angelina Tram.
Bryan Kw

In [21]:
print(no_preferences_student)

['Aashna Sibal', 'Shivani Kolla', 'Candace Baik', 'Serena Kim', 'Sophia Coulsell', 'Shloak Rathod', 'Brandon Lindquist', 'Cindy Bassil', 'Michelle Lee', 'Ashley Espinoza', 'Isabelle Hong', 'Warren Wu', 'Sirui Kang', 'Akshansh Khrodia', 'Vincent Lin', 'Nikita Senthil', 'Amber Yeung', 'Alyssa Adriano', 'Justin Louie', 'Amy Zhao', 'Ao Jiao', 'Laura Lu', 'Tina Ma', 'Kyle Kotanchek', 'Ashley Wu', 'Harper Dunn', 'Tania Firouzabady', 'Crystal Chan', 'Giovanni Zarich', 'Impana Chimmalagi', 'Lillian Gonick', 'Ria Sanghera', 'Khanh Tran', 'Ahuvah Liebman', 'Jessica Tsubamoto', 'Megan Gray', 'Luz Duran', 'Lydia Huang', 'Anusha Lingampally', 'Iryna Iziumska', 'Alina Lambacher', 'Katherine Lim', 'Sahiti Gabrani', 'Anika Malayappan', 'Anthony Chen', 'Melina Eftekhari', 'Tiffany Peverilla', 'Melissa Rupert', 'Aracely Alvarenga', 'Sourish Saswade', 'Michelle Haung', 'Jennie Ren', 'Balin Patel', 'Kana Park', 'Priya Shah', 'Kelly Yu', 'Michael Jiang', 'Emilia Leverick', 'Felix Qu', 'Audrey Sogata', 'Gay

In [22]:
#idea ---
''''
go through each student in no_prefernce list
go through each company
  - see if that company is full
if the company is not full
  - and that students major doesnt appear more than 4 times
  - and that student is in that companies_info dict
  - you can add that student
elif (see if they are a match role + )
  - the case in which
else (company is full go to next company)
'''''

def match_to_first_avail():
  global matches, matched_students, info_dict, no_preferences_student, counts
  #no preferences is an ordered list so keeps order the same
  for student_name in no_preferences_student:

    student = students.loc[students[student_name_col] == student_name]
    student_major = student[student_major_col].iloc[0] #without iloc you get the pandas series. with iloc[0] you get value without metadata

    #go through each company and try to see if you can match them there
    for company_index, company in companies.iterrows():
      company_name = company[companies_name_col].strip()
      if len(matches[company_name]) <= 7:
        matches[company_name].append(student_name)
        matched_students.add(student_name)
        student_match = True
        break

match_to_first_avail()

In [23]:
for match_list in matches.keys():
  print(match_list , " " , len(matches[match_list]), " ", matches[match_list])

PPG Industries   8   ['Althea Mendoza', 'Khanakk Garg', 'Abigail Jin', 'Aashna Sibal', 'Shivani Kolla', 'Candace Baik', 'Serena Kim', 'Sophia Coulsell']
Cadence   8   ['Sophie Zhu', 'Lindsay Qin', 'Ryan Yeo', 'Summer Dixon', 'Roocha Thatte', 'Renuka Bhusari', 'Victor Lee', 'James He']
Los Angeles Department of Water and Power (LADWP)   8   ['Elisha Reyes', 'Sidney Sabater', 'Tina Fang', 'Jason Wright', 'Shloak Rathod', 'Brandon Lindquist', 'Cindy Bassil', 'Michelle Lee']
DIRECTV   8   ['Riya Gupta', 'Sharon Chen', 'Angela Tan', 'Colleen Lam', 'Michael Bunte', 'Anusha Singhai', 'Jasmine Wongphatarakul', 'Corey Shen']
Siemens DISW   8   ['Leah George', 'Khush Parikh', 'Samuel Lu', 'Alex Yoon', 'May He', 'Tiffany Chen', 'Ashley Cheng', 'Tiffany Orian']
The Aerospace Corporation   8   ['Louis Law', 'Nandana Venkitesh', 'Anika Misra', 'Janet Liu', 'Ajay Patel', 'Anne Do', 'Lynnett Truong', 'Olivia-Emerson Ford']
Northrop Grumman   8   ['Claris Chan', 'Kyla Letko', 'Clara Lee', 'Finnley Hart

In [24]:
print(matches.items())

dict_items([('PPG Industries', ['Althea Mendoza', 'Khanakk Garg', 'Abigail Jin', 'Aashna Sibal', 'Shivani Kolla', 'Candace Baik', 'Serena Kim', 'Sophia Coulsell']), ('Cadence', ['Sophie Zhu', 'Lindsay Qin', 'Ryan Yeo', 'Summer Dixon', 'Roocha Thatte', 'Renuka Bhusari', 'Victor Lee', 'James He']), ('Los Angeles Department of Water and Power (LADWP)', ['Elisha Reyes', 'Sidney Sabater', 'Tina Fang', 'Jason Wright', 'Shloak Rathod', 'Brandon Lindquist', 'Cindy Bassil', 'Michelle Lee']), ('DIRECTV', ['Riya Gupta', 'Sharon Chen', 'Angela Tan', 'Colleen Lam', 'Michael Bunte', 'Anusha Singhai', 'Jasmine Wongphatarakul', 'Corey Shen']), ('Siemens DISW', ['Leah George', 'Khush Parikh', 'Samuel Lu', 'Alex Yoon', 'May He', 'Tiffany Chen', 'Ashley Cheng', 'Tiffany Orian']), ('The Aerospace Corporation', ['Louis Law', 'Nandana Venkitesh', 'Anika Misra', 'Janet Liu', 'Ajay Patel', 'Anne Do', 'Lynnett Truong', 'Olivia-Emerson Ford']), ('Northrop Grumman', ['Claris Chan', 'Kyla Letko', 'Clara Lee', 'Fi

In [26]:
# Convert the dictionary to a DataFrame
matches_df = pd.DataFrame.from_dict(matches, orient='index')

# Reset the index so that the company name becomes a column
matches_df = matches_df.reset_index()

# Rename the columns
matches_df.columns = ['Company Name']  + [f'Student {i+1}' for i in range(matches_df.shape[1] - 1)]
matches_df.insert(1, 'Company Representative 1', [name for name in rep_names[first_rep_column]])
matches_df.insert(2, 'Company Representative 2', [name for name in rep_names[second_rep_column]] )



import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']


gc = gspread.authorize(creds)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# open a google sheet
gs = gc.open_by_key('1i1NSwArij-rHjpsalf6Ri3ERYTnaM5CJtGuVpDVNnL8')
# select a work sheet from its name
worksheet1 = gs.worksheet('Sheet1')


worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=matches_df, include_index=False,
include_column_header=True, resize=True)
